In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy.stats import binned_statistic_2d
%matplotlib inline

In [3]:
ny_coordinates = [[ -74.25559, -73.70001],[40.91553, 40.49612]]
NY_west = ny_coordinates[0][0]
NY_east = ny_coordinates[0][1]
NY_north = ny_coordinates[1][0]
NY_south =  ny_coordinates[1][1]

In [4]:
# Берем поездки и считываем функцией binned_stat для получения данных по поездкам
def trips_2d_stat (x,y, marked_regions, crd = [-74.25559, -73.70001, 40.49612, 40.91553]):
    west,east,south,north=crd
    stat = binned_statistic_2d(x,y, np.arange(len(x)), statistic='count',
                              range=[[west,east],[south,north]],
                              bins=[marked_regions.west.append(marked_regions.east).unique(),
                                   marked_regions.south.append(marked_regions.north).unique()],
                              expand_binnumbers=True)
    return(stat.statistic.ravel())
def hour(date):
    date = date.replace(day = 1, hour = 0, minute = 0, second =0)
    month_target = date.month
    month = date.month
    result =[]
    while month_target == month:
        result.append(date)
        date = date + datetime.timedelta(hours = 1)
        month = date.month
    return(result)
#function for finding long-latit region
def reg_l_l(reg_data, data):
    print(data)
    return((reg_data.north >= data.pickup_latitude)& # Ю или равен С
          (reg_data.south <= data.pickup_latitude)&  # С или равен Ю
          (reg_data.west  <= data.pickup_longitude)& # В или равен З
          (reg_data.east  >=  data.pickup_longitude))# З или равен В
# делим сектора
def long(val, west, east):
#     return(abs(np.ceil(((val-west)*50)/(east-west))))
    return(np.ceil(((val-west)*50)/(east-west)))
def lat(val, north, south):
#     return(abs(np.ceil(((val-south)*50)/(north-south))))
    return(np.ceil(((val-south)*50)/(north-south)))
def sector(latitude, longitude, north, south, east, west):
    return((long(longitude, west, east) -1)*50 + lat(latitude, north, south))
#assert regions[reg_l_l(regions, data.iloc[0])].region.values[0] == sector(data.iloc[0].pickup_latitude, data.iloc[0].pickup_longitude, NY_north, NY_south, NY_east, NY_west),'Result from *finding sector* function isn`t equal'
#print('Checking complete. All is ok.')

In [17]:
%%time

arr = []
files = ['yellow_tripdata_2015-04.csv','yellow_tripdata_2015-05.csv','yellow_tripdata_2015-06.csv','yellow_tripdata_2015-07.csv','yellow_tripdata_2015-08.csv','yellow_tripdata_2015-09.csv','yellow_tripdata_2015-10.csv','yellow_tripdata_2015-11.csv']
for i in range(len(files)):
    data = pd.read_csv(files[i], header=0, skipinitialspace=True,
                  parse_dates=[1,2],
                  date_parser= lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    regions = pd.read_csv('regions.csv', sep=';')
    # %%time 
    #data = apr_15
    # filter data
    data = data[(data.tpep_pickup_datetime != data.tpep_dropoff_datetime)&
                (data.passenger_count > 0)&
                (data.trip_distance > 0)&
                (data.pickup_latitude <= NY_north)&
                (data.pickup_latitude >= NY_south)&
                (data.pickup_longitude<= NY_east)&
                (data.pickup_longitude>= NY_west)
               ]
    #datetime correct
    data.tpep_pickup_datetime = data.tpep_pickup_datetime.apply(lambda x: x.replace(minute=0, second=0))
    #create region
    data['region'] = sector(data.pickup_latitude, data.pickup_longitude, NY_north, NY_south, NY_east, NY_west)
    #take hours
    hours = hour(data.tpep_pickup_datetime[0])
    for i in hours:
        values = np.zeros((regions.shape[0],))
        regions[i] = values
    # stats at trips
    for i in set(data.tpep_pickup_datetime):
        chunk = data[data.tpep_pickup_datetime == i]
        x = np.array(chunk.pickup_longitude)
        y = np.array(chunk.pickup_latitude)
        regions[i] = np.array(regions[i]) + trips_2d_stat(x,y,regions)
        clmns = [0] + list(range(regions.shape[1]))[:10]
    arr.extend(regions.iloc[1282,5:])
    print('complite files:', i)


complite files: 2015-04-10 08:00:00
complite files: 2015-05-19 13:00:00
complite files: 2015-06-08 16:00:00
complite files: 2015-07-31 06:00:00
complite files: 2015-08-07 21:00:00
complite files: 2015-09-29 07:00:00
complite files: 2015-10-15 22:00:00
complite files: 2015-11-17 06:00:00
Wall time: 45min 40s


In [18]:
%%time
#arr = []
files2=['yellow_tripdata_2015-12.csv','yellow_tripdata_2015-12.csv','yellow_tripdata_2016-02.csv','yellow_tripdata_2016-03.csv','yellow_tripdata_2016-04.csv']
for i in range(len(files2)):
    data = pd.read_csv(files2[i], header=0, skipinitialspace=True,
                  parse_dates=[1,2],
                  date_parser= lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    regions = pd.read_csv('regions.csv', sep=';')
    # %%time 
    #data = apr_15
    # filter data
    data = data[(data.tpep_pickup_datetime != data.tpep_dropoff_datetime)&
                (data.passenger_count > 0)&
                (data.trip_distance > 0)&
                (data.pickup_latitude <= NY_north)&
                (data.pickup_latitude >= NY_south)&
                (data.pickup_longitude<= NY_east)&
                (data.pickup_longitude>= NY_west)
               ]
    #datetime correct
    data.tpep_pickup_datetime = data.tpep_pickup_datetime.apply(lambda x: x.replace(minute=0, second=0))
    #create region
    data['region'] = sector(data.pickup_latitude, data.pickup_longitude, NY_north, NY_south, NY_east, NY_west)
    #take hours
    hours = hour(data.tpep_pickup_datetime[0])
    for i in hours:
        values = np.zeros((regions.shape[0],))
        regions[i] = values
    # stats at trips
    for i in set(data.tpep_pickup_datetime):
        chunk = data[data.tpep_pickup_datetime == i]
        x = np.array(chunk.pickup_longitude)
        y = np.array(chunk.pickup_latitude)
        regions[i] = np.array(regions[i]) + trips_2d_stat(x,y,regions)
        clmns = [0] + list(range(regions.shape[1]))[:10]
    arr.extend(regions.iloc[1282,5:])
    print('complite files:', i)


complite files: 2015-12-03 04:00:00
complite files: 2015-12-03 04:00:00
complite files: 2016-02-25 06:00:00
complite files: 2016-03-27 16:00:00
complite files: 2016-04-10 09:00:00
Wall time: 27min 43s


In [20]:
len(arr)

9504

In [21]:
arr_cp = arr

In [49]:
ind = pd.date_range(start='2015-04-01-00-00-00', end='2016-04-30-23-00-00',periods=9504 )
len(ind)
print(ind)

DatetimeIndex(['2015-04-01 00:00:00+00:00', '2015-04-01 01:00:00+00:00',
               '2015-04-01 02:00:00+00:00', '2015-04-01 03:00:00+00:00',
               '2015-04-01 04:00:00+00:00', '2015-04-01 05:00:00+00:00',
               '2015-04-01 06:00:00+00:00', '2015-04-01 07:00:00+00:00',
               '2015-04-01 08:00:00+00:00', '2015-04-01 09:00:00+00:00',
               ...
               '2016-04-30 14:00:00+00:00', '2016-04-30 15:00:00+00:00',
               '2016-04-30 16:00:00+00:00', '2016-04-30 17:00:00+00:00',
               '2016-04-30 18:00:00+00:00', '2016-04-30 19:00:00+00:00',
               '2016-04-30 20:00:00+00:00', '2016-04-30 21:00:00+00:00',
               '2016-04-30 22:00:00+00:00', '2016-04-30 23:00:00+00:00'],
              dtype='datetime64[ns, tzutc()]', length=9504, freq=None)


In [50]:
arr_cp = np.asanyarray(arr_cp)
#ind = pd.date_range(start='2015-04-01-00-00-00', end='2016-04-30-23-00-00')
ind = pd.date_range(start='2015-04-01-00-00-00', end='2016-04-30-23-00-00',periods=9504 )
df = pd.DataFrame(arr_cp, index=ind)
df = df.T
with open('region_1283.pkl', 'wb') as file:
    pickle.dump(df, file)
print('File region_1283.pkl is ready')

File region_1283.pkl is ready


In [52]:
regions.iloc[1283]

region                 1284.000000
west                    -73.977800
east                    -73.966688
south                    40.772931
north                    40.781319
                          ...     
2016-04-30 19:00:00     164.000000
2016-04-30 20:00:00     102.000000
2016-04-30 21:00:00      75.000000
2016-04-30 22:00:00      97.000000
2016-04-30 23:00:00      66.000000
Name: 1283, Length: 725, dtype: float64

In [108]:
regions = pd.read_csv('regions.csv', sep=';')

In [ ]:
m = []
for i in range(len(regions.region)):
    regions.iloc[i,5:].values.sum/len(regions.iloc[i,5:])

In [114]:
len(regions.iloc[0,5:])

0

In [118]:
regions['sum_trips'] = regions[list(regions.iloc[:,5:])].sum(axis=1)


In [69]:
regions['sum_trips'] = 

1

In [72]:
summ = []
for i in range(len(regions.region)):
    summ.append(regions.iloc[i,5:])


In [128]:
regions.sum_trips[0]

0.0

In [129]:
regions.sum_trips.min()

0.0

In [86]:
regions['summ']= summ

In [97]:
reg = regions['summ'].

In [124]:
for i in range(len(regions.region)):
    if regions.sum_trips == regions.sum_trips.max():
        print(regions.region[i])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [123]:
regions.sum_trips.max()

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>

In [121]:
regions

,region,west,east,south,north,sum_trips
0,1,-74.255590,-74.244478,40.496120,40.504508,0.0
1,2,-74.255590,-74.244478,40.504508,40.512896,0.0
2,3,-74.255590,-74.244478,40.512896,40.521285,0.0
3,4,-74.255590,-74.244478,40.521285,40.529673,0.0
4,5,-74.255590,-74.244478,40.529673,40.538061,0.0
...,...,...,...,...,...,...
2495,2496,-73.711122,-73.700010,40.873589,40.881977,0.0
2496,2497,-73.711122,-73.700010,40.881977,40.890365,0.0
2497,2498,-73.711122,-73.700010,40.890365,40.898754,0.0
2498,2499,-73.711122,-73.700010,40.898754,40.907142,0.0


In [ ]:
# df = df.T
# with open('region_1283.pkl', 'wb') as file:
#     pickle.dump(df, file)
# print('File region_1283.pkl is ready')

In [6]:
# arr = []

In [7]:
# data = pd.read_csv(files[0], header=0, skipinitialspace=True,
#                   parse_dates=[1,2],
#                   date_parser= lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# regions = pd.read_csv('regions.csv', sep=';')
# # %%time 
# #data = apr_15
# # filter data
# data = data[(data.tpep_pickup_datetime != data.tpep_dropoff_datetime)&
#             (data.passenger_count > 0)&
#             (data.trip_distance > 0)&
#             (data.pickup_latitude <= NY_north)&
#             (data.pickup_latitude >= NY_south)&
#             (data.pickup_longitude<= NY_east)&
#             (data.pickup_longitude>= NY_west)
#            ]
# #datetime correct
# data.tpep_pickup_datetime = data.tpep_pickup_datetime.apply(lambda x: x.replace(minute=0, second=0))
# #create region
# data['region'] = sector(data.pickup_latitude, data.pickup_longitude, NY_north, NY_south, NY_east, NY_west)
# #take hours
# hours = hour(data.tpep_pickup_datetime[0])
# for i in hours:
#     values = np.zeros((regions.shape[0],))
#     regions[i] = values
# # stats at trips
# for i in set(data.tpep_pickup_datetime):
#     chunk = data[data.tpep_pickup_datetime == i]
#     x = np.array(chunk.pickup_longitude)
#     y = np.array(chunk.pickup_latitude)
#     regions[i] = np.array(regions[i]) + trips_2d_stat(x,y,regions)
#     clmns = [0] + list(range(regions.shape[1]))[:10]
# arr.extend(regions.iloc[1282,5:])
# #print('complite files:'% i)

In [8]:
# data = pd.read_csv(files[1], header=0, skipinitialspace=True,
#                   parse_dates=[1,2],
#                   date_parser= lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# regions = pd.read_csv('regions.csv', sep=';')
# # %%time 
# #data = apr_15
# # filter data
# data = data[(data.tpep_pickup_datetime != data.tpep_dropoff_datetime)&
#             (data.passenger_count > 0)&
#             (data.trip_distance > 0)&
#             (data.pickup_latitude <= NY_north)&
#             (data.pickup_latitude >= NY_south)&
#             (data.pickup_longitude<= NY_east)&
#             (data.pickup_longitude>= NY_west)
#            ]
# #datetime correct
# data.tpep_pickup_datetime = data.tpep_pickup_datetime.apply(lambda x: x.replace(minute=0, second=0))
# #create region
# data['region'] = sector(data.pickup_latitude, data.pickup_longitude, NY_north, NY_south, NY_east, NY_west)
# #take hours
# hours = hour(data.tpep_pickup_datetime[0])
# for i in hours:
#     values = np.zeros((regions.shape[0],))
#     regions[i] = values
# # stats at trips
# for i in set(data.tpep_pickup_datetime):
#     chunk = data[data.tpep_pickup_datetime == i]
#     x = np.array(chunk.pickup_longitude)
#     y = np.array(chunk.pickup_latitude)
#     regions[i] = np.array(regions[i]) + trips_2d_stat(x,y,regions)
#     clmns = [0] + list(range(regions.shape[1]))[:10]
# arr.extend(regions.iloc[1282,5:])
# #    print('complite files:'% i)

In [9]:
# arr

[103.0,
 31.0,
 4.0,
 7.0,
 5.0,
 26.0,
 77.0,
 250.0,
 360.0,
 334.0,
 362.0,
 364.0,
 398.0,
 429.0,
 379.0,
 369.0,
 312.0,
 371.0,
 435.0,
 435.0,
 353.0,
 348.0,
 280.0,
 232.0,
 124.0,
 74.0,
 17.0,
 14.0,
 8.0,
 25.0,
 79.0,
 227.0,
 326.0,
 293.0,
 352.0,
 345.0,
 400.0,
 424.0,
 444.0,
 383.0,
 271.0,
 328.0,
 402.0,
 357.0,
 299.0,
 278.0,
 184.0,
 167.0,
 125.0,
 53.0,
 25.0,
 22.0,
 17.0,
 35.0,
 43.0,
 119.0,
 176.0,
 262.0,
 363.0,
 381.0,
 403.0,
 353.0,
 314.0,
 325.0,
 284.0,
 366.0,
 361.0,
 279.0,
 197.0,
 236.0,
 216.0,
 185.0,
 149.0,
 61.0,
 25.0,
 11.0,
 18.0,
 7.0,
 26.0,
 50.0,
 83.0,
 220.0,
 261.0,
 332.0,
 392.0,
 442.0,
 373.0,
 436.0,
 370.0,
 428.0,
 453.0,
 376.0,
 256.0,
 206.0,
 206.0,
 231.0,
 104.0,
 82.0,
 32.0,
 23.0,
 27.0,
 21.0,
 28.0,
 57.0,
 91.0,
 133.0,
 193.0,
 267.0,
 342.0,
 375.0,
 382.0,
 415.0,
 346.0,
 278.0,
 296.0,
 204.0,
 176.0,
 116.0,
 81.0,
 62.0,
 22.0,
 9.0,
 4.0,
 3.0,
 12.0,
 18.0,
 67.0,
 166.0,
 228.0,
 247.0,
 275.0,
 33

In [79]:
# regions = pd.read_csv('regions.csv', sep=';')

In [13]:
# with open('all_regions.pkl','rb') as file:
#     simple = pickle.load(file, encoding='latin1')

In [300]:
# %%time 
# apr_15 = pd.read_csv('yellow_tripdata_2015-04.csv', header=0, skipinitialspace=True,
#                   parse_dates=[1,2],
#                   date_parser= lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [302]:
# file0 = 'yellow_tripdata_2015-04.csv'
# file1 = 'yellow_tripdata_2015-05.csv'
# file2 = 'yellow_tripdata_2015-06.csv'
# file3 = 'yellow_tripdata_2015-07.csv'
# file4 = 'yellow_tripdata_2015-08.csv'
# file5 = 'yellow_tripdata_2015-09.csv'
# file6 = 'yellow_tripdata_2015-10.csv'
# file7 = 'yellow_tripdata_2015-11.csv'
# file8 = 'yellow_tripdata_2015-12.csv'
# file9 = 'yellow_tripdata_2016-01.csv'
# file10 = 'yellow_tripdata_2016-02.csv'
# file11 = 'yellow_tripdata_2016-03.csv'
# file12 = 'yellow_tripdata_2016-04.csv'

In [6]:
data = pd.read_csv('yellow_tripdata_2016-01.csv', header=0, skipinitialspace=True,
                  parse_dates=[1,2],
                  date_parser= lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
regions = pd.read_csv('regions.csv', sep=';')


In [7]:
data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01 00:00:00,2016-01-01 00:00:00,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0.00,0.00,0.3,8.80
1,2,2016-01-01 00:00:00,2016-01-01 00:00:00,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0.00,0.00,0.3,19.30
2,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,33.0,0.5,0.5,0.00,0.00,0.3,34.30
3,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,16.5,0.0,0.5,0.00,0.00,0.3,17.30
4,2,2016-01-01 00:00:00,2016-01-01 00:00:00,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,8.0,0.0,0.5,0.00,0.00,0.3,8.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10906853,2,2016-01-31 23:30:32,2016-01-31 23:38:18,1,2.20,-74.003578,40.751011,1,N,-73.982651,40.767509,2,8.5,0.5,0.5,0.00,0.00,0.3,9.80
10906854,1,2016-01-05 00:15:55,2016-01-05 00:16:06,1,0.00,-73.945488,40.751530,1,N,-73.945457,40.751530,2,2.5,0.5,0.5,0.00,0.00,0.3,3.80
10906855,1,2016-01-05 06:12:46,2016-03-19 20:45:50,3,1.40,-73.994240,40.766586,1,N,-73.984428,40.753922,2,7.5,0.5,0.5,0.00,0.00,0.3,8.80
10906856,1,2016-01-05 06:21:44,2016-03-28 12:54:26,1,2.10,-73.948067,40.776531,1,N,-73.978188,40.777435,1,11.5,0.0,0.5,2.45,0.00,0.3,14.75


In [ ]:
# %%time 
#data = apr_15
# filter data
data = data[(data.tpep_pickup_datetime != data.tpep_dropoff_datetime)&
            (data.passenger_count > 0)&
            (data.trip_distance > 0)&
            (data.pickup_latitude <= NY_north)&
            (data.pickup_latitude >= NY_south)&
            (data.pickup_longitude<= NY_east)&
            (data.pickup_longitude>= NY_west)
           ]
#datetime correct
data.tpep_pickup_datetime = data.tpep_pickup_datetime.apply(lambda x: x.replace(minute=0, second=0))
#create region
data['region'] = sector(data.pickup_latitude, data.pickup_longitude, NY_north, NY_south, NY_east, NY_west)
#take hours
hours = hour(data.tpep_pickup_datetime[0])
for i in hours:
    values = np.zeros((regions.shape[0],))
    regions[i] = values
# stats at trips
for i in set(data.tpep_pickup_datetime):
    chunk = data[data.tpep_pickup_datetime == i]
    x = np.array(chunk.pickup_longitude)
    y = np.array(chunk.pickup_latitude)
    regions[i] = np.array(regions[i]) + trips_2d_stat(x,y,regions)
    clmns = [0] + list(range(regions.shape[1]))[:10]
arr.extend(regions.iloc[1282,5:])
#    print('complite files:'% i)